In [1]:
import pandas as pd

In [3]:
filename = 'APPLE_20130101_20131231.csv'
datas = pd.read_csv('./tmp/' + filename)

In [4]:
import os
import sys
import nltk

if os.path.exists(os.path.abspath('./finbert_model/pytorch_model.bin')) == False:
    os.system('git lfs install')
    os.system('git clone https://huggingface.co/ProsusAI/finbert finbert_model')

finbert_path = os.path.abspath('./finbert')
if (finbert_path in sys.path) == False:
    sys.path.append(finbert_path)

nltk.download('punkt')

c:\Workspace\80_Framework\anaconda\envs\FinBERT\lib\site-packages\scipy\__init__.py:149: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.16.3
  UserWarning)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xenos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from finbert.finbert import predict
import finbert.utils as utils
from transformers import AutoModelForSequenceClassification
import logging

output_dir = './output'
if os.path.exists(output_dir) == False:
    os.mkdir(output_dir)

utils.logger.disabled = True

In [6]:

model_path = os.path.abspath('./finbert_model')
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=3, cache_dir=None)

output_path = os.path.abspath('./output/' + filename)
# os.mkdir(output_path)

In [7]:
data_column = []

for index in datas.iterrows():
    predict_result = datas
    index = datas.iloc[0]
    title = index['title']
    full_text = index['full_text']

    output = predict(full_text, model, use_gpu=True, write_to_csv=False)
    data_column.append(output)
    
print(len(data_column))

06/03/2024 00:05:12 - INFO - root -   Using device: cuda:0 
06/03/2024 00:05:12 - INFO - root -   tensor([[-0.2992,  0.0714, -0.4178],
        [-0.0680, -1.2666,  1.6589],
        [-1.7427,  2.3521, -0.1065],
        [-0.9620, -0.8915,  2.3817],
        [-1.7067,  1.5723,  1.1490]], device='cuda:0')
06/03/2024 00:05:12 - INFO - root -   tensor([[-0.4404, -1.7844,  2.4219],
        [-0.7175, -1.3861,  2.3775],
        [ 0.1146, -2.2557,  2.1051],
        [-0.3917, -1.7575,  2.4152],
        [ 1.1355, -2.5686,  0.9411]], device='cuda:0')
06/03/2024 00:05:13 - INFO - root -   tensor([[ 1.1747, -2.3589,  1.1088],
        [-1.7423,  3.0293, -0.9434],
        [-1.9306,  2.5716,  0.0264],
        [ 1.7507, -2.5075, -0.2073],
        [ 0.8383, -2.4795,  1.6527]], device='cuda:0')
06/03/2024 00:05:13 - INFO - root -   tensor([[ 2.0738, -1.8779, -1.4554],
        [-1.6041,  1.1855,  1.4632],
        [-1.5876,  2.2884, -0.2389],
        [ 0.6820,  0.4473, -2.1111],
        [-0.3116, -1.1529,  2.1

422


In [8]:
data_indices = []
for index in data_column:
    json = index.to_json()
    data_indices.append(json)

datas['inference'] = data_indices

In [9]:
datas.to_csv('./output/' + filename)